In [1]:
dayObs = 'Today'

In [2]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta

In [3]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')
dayObs

'20241013'

In [4]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
    'LSSTCam/calib/unbounded',
    'LSSTCam/photodiode'
])

In [5]:
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    hasPD=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, hasPD])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            record.obs_id,
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            hasPD,
            f'http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id}'
                     )
            for [id,record,nimage, hasPD] in exposureList ], 
            names=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'hasPD', 'ImageViewer'
                  ])
table.pprint_all()

       obs_id                   time            seqnum observation_type science_program observation_reason exposure_time physical_filter nimage hasPD                                 ImageViewer                                 
-------------------- -------------------------- ------ ---------------- --------------- ------------------ ------------- --------------- ------ ----- ----------------------------------------------------------------------------
MC_C_20241013_000001 2024-10-13 12:28:14.578014      1             bias            E756           preimage           0.0         unknown    197     0 http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image=MC_C_20241013_000001
MC_C_20241013_000002 2024-10-13 12:28:17.169994      2             bias            E756           preimage           0.0         unknown    197     0 http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image=MC_C_20241013_000002
MC_C_20241013_000003 2024-10-13 12:28:20.108984      3             bias            E756     